In [18]:
from data_source import MarketPriceData
import MetaTrader5 as mt5

# Broker credentials
login = 51439076
server = "ICMarketsSC-Demo"
password = "vwYYuaZt"
fetcher = MarketPriceData(login=login, server=server, password=password)

# Choose asset class, timeframe, date range, and number of rows to be fetched
asset = "EURUSD"
timeframe = mt5.TIMEFRAME_M1
date_from = None
bars = 40000

# Print data
data = fetcher.fetch_data(asset=asset, timeframe=timeframe, date_from=date_from, bars=bars)
data.drop("real_volume", axis=1, inplace=True)

# Adding Technical Indicator Data

In [19]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import talib

data.rename(columns={'tick_volume':'volume'}, inplace=True)
data.set_index(pd.DatetimeIndex(data["time"]), inplace=True)
data.ta.strategy(ta.AllStrategy, verbose=True, timed=True)

data = data.drop('time', axis=1)

[+] Strategy: All
[i] Indicator arguments: {'timed': True, 'append': True}
[i] Excluded[12]: above, above_value, below, below_value, cross, cross_value, long_run, short_run, td_seq, tsignals, vp, xsignals
[i] Multiprocessing 131 indicators with 3 chunks and 8/8 cpus.
[i] Total indicators: 131
[i] Columns added: 278
[i] Last Run: Friday November 10, 2023, NYSE: 2:46:31, Local: 6:46:31 Eastern Standard Time, Day 314/365 (86.00%)
[i] Runtime: 31873.6786 ms (31.8737 s)


In [12]:
data

,time,open,high,low,close,tick_volume,spread
0,2023-10-03 14:23:00,1.04738,1.04742,1.04726,1.04730,47,0
1,2023-10-03 14:24:00,1.04730,1.04730,1.04698,1.04698,81,0
2,2023-10-03 14:25:00,1.04698,1.04731,1.04698,1.04714,46,0
3,2023-10-03 14:26:00,1.04714,1.04729,1.04710,1.04727,51,0
4,2023-10-03 14:27:00,1.04727,1.04735,1.04707,1.04708,47,0
...,...,...,...,...,...,...,...
39995,2023-11-10 09:43:00,1.06659,1.06686,1.06656,1.06682,64,0
39996,2023-11-10 09:44:00,1.06681,1.06684,1.06665,1.06684,68,0
39997,2023-11-10 09:45:00,1.06683,1.06694,1.06681,1.06691,42,0
39998,2023-11-10 09:46:00,1.06691,1.06697,1.06682,1.06683,50,0


In [20]:
threshold = len(data) * 0.80
data = data.dropna(axis='columns', thresh=threshold)

In [21]:
# Function to replace NaNs and infinite values with the mean for each column
def replace_with_mean(df):
    for column in df.columns:
        if df[column].dtype.kind in 'fc':  # f for float and c for complex numbers
            # Calculate mean without considering inf values
            mean_val = df[column].replace([np.inf, -np.inf], np.nan).mean()
            # Replace NaNs and infs with the calculated mean
            df[column] = df[column].replace([np.inf, -np.inf], mean_val).fillna(mean_val)
    return df

# Apply the function to the 'data' dataframe
data = replace_with_mean(data)


In [22]:
print(data.head)

<bound method NDFrame.head of                         open     high      low    close  volume  spread  \
time                                                                      
2023-10-03 16:21:00  1.04700  1.04706  1.04696  1.04701      43       0   
2023-10-03 16:22:00  1.04700  1.04709  1.04698  1.04709      36       0   
2023-10-03 16:23:00  1.04708  1.04714  1.04698  1.04714      32       0   
2023-10-03 16:24:00  1.04714  1.04715  1.04700  1.04712      38       0   
2023-10-03 16:25:00  1.04712  1.04726  1.04712  1.04712      43       0   
...                      ...      ...      ...      ...     ...     ...   
2023-11-10 11:41:00  1.06696  1.06698  1.06692  1.06698      23       0   
2023-11-10 11:42:00  1.06698  1.06706  1.06694  1.06706      40       0   
2023-11-10 11:43:00  1.06706  1.06711  1.06704  1.06706      25       0   
2023-11-10 11:44:00  1.06707  1.06726  1.06706  1.06725      34       0   
2023-11-10 11:45:00  1.06725  1.06749  1.06723  1.06749      48       

In [6]:
data.to_csv('eurusd_small_ta.csv', index=True)

# Target Variable

In [39]:
from data_source import MarketPriceData
import MetaTrader5 as mt5
from datetime import datetime


# Broker credentials
login = 51439076
server = "ICMarketsSC-Demo"
password = "vwYYuaZt"
fetcher = MarketPriceData(login=login, server=server, password=password)

# Choose asset class, timeframe, date range, and number of rows to be fetched

utc_from = datetime(2023, 11, 9, 20, 22) # local machine time

asset = "EURUSD"
timeframe = mt5.TIMEFRAME_M1
date_from = utc_from
bars = 10000

# Print data
data = fetcher.fetch_data(asset=asset, timeframe=timeframe, date_from=date_from, bars=bars)
data.drop(['real_volume', 'spread', 'tick_volume'], axis=1, inplace=True)


import pandas as pd
import numpy as np

def identify_candlestick(df):
    # Define the threshold for a 'little to no shadow' as a percentage of the candle range
    shadow_threshold = 0.05
    
    # Define the threshold for a doji to be considered as having approximately equal open and close
    doji_threshold = (df['high'] - df['low']) * 0.1

    # Initialize the candlestick pattern column to 'Neutral'
    df['candlestick_pattern'] = 'Neutral'

    # Calculate the necessary values within the function scope
    body = abs(df['close'] - df['open'])
    upper_shadow = df['high'] - df[['open', 'close']].max(axis=1)
    lower_shadow = df[['open', 'close']].min(axis=1) - df['low']

    # Conditions for candlestick patterns
    bullish_marubozu_cond = ((df['close'] > df['open']) & 
                              (upper_shadow <= shadow_threshold * body) & 
                              (lower_shadow <= shadow_threshold * body))

    bearish_marubozu_cond = ((df['open'] > df['close']) & 
                              (upper_shadow <= shadow_threshold * body) & 
                              (lower_shadow <= shadow_threshold * body))

    bullish_doji_cond = (abs(df['close'] - df['open']) <= doji_threshold) & (df['close'] < df['open'])
    
    bearish_doji_cond = (abs(df['close'] - df['open']) <= doji_threshold) & (df['close'] > df['open'])

    # Assigning candlestick patterns
    df.loc[bullish_marubozu_cond, 'candlestick_pattern'] = 'Bullish Marubozu'
    df.loc[bearish_marubozu_cond, 'candlestick_pattern'] = 'Bearish Marubozu'
    df.loc[bullish_doji_cond, 'candlestick_pattern'] = 'Bullish Doji'
    df.loc[bearish_doji_cond, 'candlestick_pattern'] = 'Bearish Doji'
    df.loc[(df['close'] > df['open']) & ~(bullish_marubozu_cond | bullish_doji_cond), 'candlestick_pattern'] = 'Regular Bullish'
    df.loc[(df['open'] > df['close']) & ~(bearish_marubozu_cond | bearish_doji_cond), 'candlestick_pattern'] = 'Regular Bearish'

    return df



target_variable = identify_candlestick(data)
print(data)

                    time     open     high      low    close  \
0    2023-11-01 02:34:00  1.05780  1.05780  1.05774  1.05774   
1    2023-11-01 02:35:00  1.05774  1.05778  1.05773  1.05776   
2    2023-11-01 02:36:00  1.05776  1.05776  1.05776  1.05776   
3    2023-11-01 02:37:00  1.05776  1.05776  1.05776  1.05776   
4    2023-11-01 02:38:00  1.05776  1.05792  1.05775  1.05786   
...                  ...      ...      ...      ...      ...   
9995 2023-11-10 01:18:00  1.06662  1.06664  1.06654  1.06654   
9996 2023-11-10 01:19:00  1.06654  1.06654  1.06654  1.06654   
9997 2023-11-10 01:20:00  1.06654  1.06655  1.06653  1.06654   
9998 2023-11-10 01:21:00  1.06654  1.06654  1.06646  1.06646   
9999 2023-11-10 01:22:00  1.06646  1.06651  1.06645  1.06649   

     candlestick_pattern  
0       Bearish Marubozu  
1        Regular Bullish  
2                Neutral  
3                Neutral  
4        Regular Bullish  
...                  ...  
9995     Regular Bearish  
9996           

In [40]:
target_variable['candlestick_pattern'].value_counts()

Regular Bullish     4081
Regular Bearish     3972
Neutral              882
Bullish Marubozu     561
Bearish Marubozu     504
Name: candlestick_pattern, dtype: int64

In [25]:
target_variable.to_csv('target_variable.csv')